# Resolução Exercício 02

**Aluno**: Leonardo de Andrade Santana <br>
**Disciplina**: Inteligência Artificial

In [53]:
import numpy as np
import random

LOW_BATCH_SIZE = 4
HIGH_BATCH_SIZE = 129

batch_size = range(LOW_BATCH_SIZE, HIGH_BATCH_SIZE)
base_model = ['VGG16', 'VGG19', 'DenseNet121', 'ResNet50', 'InceptionV3', 'InceptionResNetV2', 'NASNetMobile', 'NASNetLarge']
image_dimension = [224,299,448,598]
optimizer = ['SGD','RMSprop','Adam','Adadelta','Adagrad','Adamax','Nadam','Ftrl']

NUM_GENERATIONS = 10
NUM_INDIVIDUALS = 10
MAX_BEST_INDIVIDUALS = 3
CHANCE_MUT = .20
CHANCE_CO = .25

A sequência de loops a seguir estava com um dos `for` fora do outro, o que causava o resultado de 4000 casos, mas que são na verdade 32000.

In [17]:
import pandas as pd
import numpy as np
import random
df = pd.DataFrame(columns=['batch_size', 'base_model', 'image_dimension', 'optimizer', 'loss_rate'])

for batch in batch_size:
    loss_rate1 = np.random.uniform(0, 1)
    for model in base_model:
        loss_rate2 = loss_rate1 + np.random.uniform(0, 1)
        for image in image_dimension:
            loss_rate3 = loss_rate2 +np.random.uniform(0, 1)
            for opt in optimizer:
                loss_rate = loss_rate3 + np.random.uniform(0, 1)
                df.loc[len(df)] = [batch, model, image, opt, loss_rate]

df

,batch_size,base_model,image_dimension,optimizer,loss_rate
0,4,VGG16,224,SGD,1.965571
1,4,VGG16,224,RMSprop,1.723888
2,4,VGG16,224,Adam,1.641528
3,4,VGG16,224,Adadelta,1.648850
4,4,VGG16,224,Adagrad,1.354461
...,...,...,...,...,...
31995,128,NASNetLarge,598,Adadelta,1.750488
31996,128,NASNetLarge,598,Adagrad,2.208005
31997,128,NASNetLarge,598,Adamax,1.365511
31998,128,NASNetLarge,598,Nadam,1.887733


In [18]:
def random_population(n):
    population = []
    for i in range(n):
        population.append([random.randint(4, 129), random.choice(base_model), random.choice(image_dimension), random.choice(optimizer)])
    return population

In [23]:
def load_fitness(ind):
    for index, row in df.iterrows():
        if row['batch_size'] == ind[0] and row['base_model'] == ind[1] and row['image_dimension'] == ind[2] and row['optimizer'] == ind[3]:
            return row['loss_rate']

In [48]:
def mutate_batch_size(batch_size):
    new_batch_size = batch_size
    new_batch_size += random.randint(-3, 3)

    if new_batch_size >= LOW_BATCH_SIZE and new_batch_size <= HIGH_BATCH_SIZE: return new_batch_size
    else: mutate_batch_size(batch_size)

def mutate(ind):
    
    # Mutate the batch_size value
    if np.random.uniform(0, 1) < CHANCE_MUT:
        ind[0] = mutate_batch_size(ind[0])
    
    # Mutate the base model value
    if np.random.uniform(0, 1) < CHANCE_MUT:
        aux = base_model.copy()
        aux.remove(ind[1])
        ind[1] = random.choice(aux)
    
    # Mutate the image_dimension value
    if np.random.uniform(0, 1) < CHANCE_MUT:
        aux = image_dimension.copy()
        aux.remove(ind[2])
        ind[2] = random.choice(aux)
    
    # Mutate the optimizer value
    if np.random.uniform(0, 1) < CHANCE_MUT:
        aux = optimizer.copy()
        aux.remove(ind[3])
        ind[3] = random.choice(aux)

In [28]:
def crossover(ind1, ind2):
    son = ind1.copy()
    for i in range(4):
        if np.random.uniform(0, 1) < CHANCE_CO:
            son[i] = ind2[i]
    return son

In [27]:
def sort_list(_list, ordi, dec=False):
    return [x for _, x in sorted(zip(ordi, _list), key=lambda p: p[0], reverse=dec)]

In [30]:
def next_generation(population, fitness):
    ord_list = sort_list(population, fitness)
    next_gen = ord_list[:MAX_BEST_INDIVIDUALS]

    while len(next_gen) < NUM_INDIVIDUALS:
        ind1, ind2 = random.choices(population, k=2)
        son = crossover(ind1, ind2)
        mutate(son)
        next_gen.append(son)

    return next_gen

In [59]:
# Main Cell -----------------------------------------------------

population = random_population(NUM_INDIVIDUALS)

print('ger | fitness\n----+-' + '-'*5*NUM_INDIVIDUALS)

for gen in range(NUM_GENERATIONS):

    fitness = []
    for ind in population:
        fitness.append(load_fitness(ind))

    population = next_generation(population, fitness)

    print('{:3} |'.format(gen), ' '.join('{:4.2f}'.format(s) for s in sorted(fitness, reverse=False)))

fitness = []
for ind in population:
    fitness.append(load_fitness(ind))
    
ord_list = sort_list(population, fitness)

best_ind = ord_list[0]
fit = load_fitness(best_ind)
fit

ger | fitness
----+---------------------------------------------------
  0 | 0.90 1.42 1.62 1.71 1.75 2.01 2.06 2.20 2.32 3.23
  1 | 0.90 1.02 1.42 1.62 1.62 1.74 1.88 1.88 2.24 2.25
  2 | 0.90 1.02 1.11 1.33 1.42 1.53 1.59 1.61 1.72 2.24
  3 | 0.90 1.02 1.11 1.50 1.73 1.97 2.14 2.18 2.29 2.39
  4 | 0.90 1.02 1.02 1.11 1.18 1.44 1.54 1.65 1.99 2.16
  5 | 0.90 1.02 1.02 1.17 1.39 1.47 1.62 2.33 2.48 2.68
  6 | 0.90 1.02 1.02 1.02 1.02 1.74 2.39 2.43 2.48 2.67
  7 | 0.72 0.90 0.90 1.02 1.02 1.48 1.51 1.79 2.43 2.48
  8 | 0.40 0.72 0.90 0.90 0.96 1.51 1.67 1.75 2.65 2.67
  9 | 0.40 0.44 0.72 0.90 1.00 1.67 1.87 2.15 2.84 3.17


0.4024558358362281